# Türkiye Konut Satış Verileri - Görselleştirme

# Giriş

Bu çalışmada TÜİK tarafından aylık olarak açıklanan konut satış verileri görselleştirilmektedir. Veriler TÜİK'in sitesinde belirli periyotlarla *excel* formatında açıklanmaktadır. Proje kapsamında görselleştirilen verilerin güncelliğini korumak ve sonradan herhangi bir düzenlemeye gerek kalmaksızın çalışmasını sağlamak amacıyla programın aylık periyotlarla otonom olarak çalışaması ve verilerin *selenium* yardımı ile çekilerek içeriğin güncellenmesi hedeflenmiştir. 

Öğrenmeye yönelik bu proje ile veri bilimi araçlarının kullanımı konusunda deneyim kazanmak aynı zamanda konut satışları ile ilgili medyada göremediğimiz detaylara görsel olarak değinilmek istenmektedir.

In [1]:
# Bağımlılıkların indirilmesi
!apt-get update -y
!apt-get install -y \
libglib2.0-0 \
libnss3 \
libdbus-glib-1-2 \
libgconf-2-4 \
libfontconfig1 \
libvulkan1 \
gconf2-common \
libwayland-server0 \
libgbm1 \
udev \
libu2f-udev 
!apt --fix-broken install -y  

Hit:1 http://archive.ubuntu.com/ubuntu focal InRelease
Get:2 http://security.ubuntu.com/ubuntu focal-security InRelease [128 kB]
Get:3 http://packages.cloud.google.com/apt gcsfuse-focal InRelease [1227 B]
Get:4 http://archive.ubuntu.com/ubuntu focal-updates InRelease [128 kB]
Get:5 https://packages.cloud.google.com/apt cloud-sdk InRelease [1618 B]
Get:6 http://archive.ubuntu.com/ubuntu focal-backports InRelease [128 kB]
Err:3 http://packages.cloud.google.com/apt gcsfuse-focal InRelease
  The following signatures couldn't be verified because the public key is not available: NO_PUBKEY C0BA5CE6DC6315A3
Get:7 http://security.ubuntu.com/ubuntu focal-security/restricted amd64 Packages [4548 kB]
Err:5 https://packages.cloud.google.com/apt cloud-sdk InRelease
  The following signatures couldn't be verified because the public key is not available: NO_PUBKEY C0BA5CE6DC6315A3
Get:8 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 Packages [4834 kB]
Get:9 http://security.ubuntu.com/ubuntu

In [2]:
# Chrome'un indirilmesi ve çıkartılması
!wget -P /tmp https://edgedl.me.gvt1.com/edgedl/chrome/chrome-for-testing/116.0.5845.96/linux64/chrome-linux64.zip
!unzip /tmp/chrome-linux64.zip -d /usr/bin/

--2025-04-22 12:03:06--  https://edgedl.me.gvt1.com/edgedl/chrome/chrome-for-testing/116.0.5845.96/linux64/chrome-linux64.zip
Resolving edgedl.me.gvt1.com (edgedl.me.gvt1.com)... 34.104.35.123, 2600:1900:4110:86f::
Connecting to edgedl.me.gvt1.com (edgedl.me.gvt1.com)|34.104.35.123|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://storage.googleapis.com/chrome-for-testing-public/116.0.5845.96/linux64/chrome-linux64.zip [following]
--2025-04-22 12:03:06--  https://storage.googleapis.com/chrome-for-testing-public/116.0.5845.96/linux64/chrome-linux64.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 192.178.163.207, 74.125.142.207, 173.194.203.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|192.178.163.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 145898081 (139M) [application/zip]
Saving to: ‘/tmp/chrome-linux64.zip’

chrome-linux64.zip  100%[===================>] 139.14M   218MB/s 

In [3]:
# Chromedriver'ın indirilmesi ve çıkartılması
!wget -P /tmp https://edgedl.me.gvt1.com/edgedl/chrome/chrome-for-testing/116.0.5845.96/linux64/chromedriver-linux64.zip
!unzip /tmp/chromedriver-linux64.zip -d /usr/bin/

--2025-04-22 12:03:13--  https://edgedl.me.gvt1.com/edgedl/chrome/chrome-for-testing/116.0.5845.96/linux64/chromedriver-linux64.zip
Resolving edgedl.me.gvt1.com (edgedl.me.gvt1.com)... 34.104.35.123, 2600:1900:4110:86f::
Connecting to edgedl.me.gvt1.com (edgedl.me.gvt1.com)|34.104.35.123|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://storage.googleapis.com/chrome-for-testing-public/116.0.5845.96/linux64/chromedriver-linux64.zip [following]
--2025-04-22 12:03:14--  https://storage.googleapis.com/chrome-for-testing-public/116.0.5845.96/linux64/chromedriver-linux64.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.99.207, 74.125.20.207, 74.125.197.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.99.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7271942 (6.9M) [application/zip]
Saving to: ‘/tmp/chromedriver-linux64.zip’

chromedriver-linux6 100%[===================>] 

In [4]:
# Selenium indirilmesi 
!apt install -y python3-selenium
!pip install selenium==3.141.0

!pip install xlrd




The following additional packages will be installed:
  apparmor chromium-browser chromium-chromedriver liblzo2-2 snapd
  squashfs-tools
Suggested packages:
  apparmor-profiles-extra apparmor-utils zenity | kdialog
The following NEW packages will be installed:
  apparmor chromium-browser chromium-chromedriver liblzo2-2 python3-selenium
  snapd squashfs-tools
0 upgraded, 7 newly installed, 0 to remove and 245 not upgraded.
Need to get 26.9 MB of archives.
After this operation, 113 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 apparmor amd64 2.13.3-7ubuntu5.4 [505 kB]
Get:2 http://archive.ubuntu.com/ubuntu focal/main amd64 liblzo2-2 amd64 2.10-2 [50.8 kB]
Get:3 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 squashfs-tools amd64 1:4.4-1ubuntu0.3 [117 kB]
Get:4 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 snapd amd64 2.67.1+20.04 [26.1 MB]
Get:5 http://archive.ubuntu.com/ubuntu focal-updates/universe amd6

In [5]:
# Kütüphanelerin içe aktarılması
import pandas as pd
import numpy as np

import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
from retrying import retry
import traceback

import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio #Plotly theming
pio.templates.default = 'plotly_white'
pio.templates["plotly_white"] = go.layout.Template(layout=go.Layout(colorway=px.colors.qualitative.G10))

# Verilerin TUİK'in sitesinden çekilmesi amacıyla kullanılması için
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains

import time
import datetime
from datetime import timedelta

#Haritalar için GeoJson bilgilerini almak için gerekli kütüphaneler
from urllib.request import urlopen 
import json

In [6]:
# Chrome Driver ayarları

CHROME_BINARY_LOCATION = "/usr/bin/chrome-linux64/chrome"
CHROMEDRIVER_BINARY_LOCATION = "/usr/bin/chromedriver-linux64/chromedriver"

def add_driver_options(options):
    """
    Add configurable options
    """
    chrome_options = Options()
    for opt in options:
        chrome_options.add_argument(opt)
    return chrome_options

def initialize_driver():
    """
    Initialize the web driver
    """
    driver_config = {
        "options": [
            "--headless",
            "--no-sandbox",
            "--start-fullscreen",
            "--allow-insecure-localhost",
            "--disable-dev-shm-usage",
            "user-agent=Chrome/116.0.5845.96"
        ],
    }
    options = add_driver_options(driver_config["options"])
    options.binary_location = CHROME_BINARY_LOCATION
    driver = webdriver.Chrome(
        executable_path=CHROMEDRIVER_BINARY_LOCATION,
        options=options)
    return driver

In [7]:
def get_category_tables(category: str, max_pages: int = 2) -> list:
    """
    Belirli bir kategorideki tabloların başlıklarını ve Excel indirme linklerini alır
    
    Args:
        category: Kategori adı (örneğin, "İnşaat ve Konut")
        max_pages: Taranacak maksimum sayfa sayısı
        
    Returns:
        Tablo başlıkları ve indirme URL'lerini içeren liste
    """
    # Kategori ID'sini al
    response = requests.get("https://data.tuik.gov.tr/")
    soup = BeautifulSoup(response.text, "html.parser")
    themes = soup.find_all("div", class_="text-center")
    
    category_id = None
    for theme in themes:
        for link in theme.find_all("a"):
            if link.text.strip() == category:
                category_id = link["href"].split("=")[-1]
                break
    
    if not category_id:
        print(f"Kategori '{category}' bulunamadı")
        return []
    
    results = []
    
    # Tarayıcı başlat
    driver = initialize_driver()
    try:
        # Kategori sayfasına git
        url = f"https://data.tuik.gov.tr/Kategori/GetKategori?p={category_id}#nav-db"
        driver.get(url)
        time.sleep(3)
        
        # İstatistiksel Tablolar sekmesine tıkla
        tables_tab = driver.find_element(By.ID, "nav-profile-tab")
        tables_tab.click()
        time.sleep(3)
        
        # Sayfa sınırlı tarama
        for page_num in range(1, max_pages + 1):
            print(f"Sayfa {page_num}/{max_pages} inceleniyor...")
            
            # Tüm satırları kontrol et
            rows = driver.find_elements(By.XPATH, "//tr")
            for row in rows:
                title_cells = row.find_elements(By.XPATH, ".//td")
                if not title_cells or len(title_cells) == 0:
                    continue
                
                current_title = title_cells[0].text.strip()
                # Başlığın boş olmadığından emin ol
                if current_title:
                    excel_links = row.find_elements(
                        By.XPATH,
                        ".//a[contains(@href, 'DownloadIstatistikselTablo')]"
                    )
                    if excel_links:
                        excel_link = excel_links[0].get_attribute("href")
                        results.append((current_title, excel_link))
            
            # Sonraki sayfa kontrolü
            if page_num < max_pages:
                next_button = driver.find_element(By.ID, "istatistikselTable_next")
                if "disabled" in next_button.get_attribute("class"):
                    print("Son sayfaya ulaşıldı.")
                    break
                
                next_button.click()
                time.sleep(3)
    
    except Exception as e:
        print(f"Bir hata oluştu: {str(e)}")
    finally:
        driver.quit()
    
    print(f"'{category}' kategorisinde {len(results)} tablo bulundu")
    return results

In [8]:
# Tablo linklerinin alınması
konut_tablolar = get_category_tables("İnşaat ve Konut")

Sayfa 1/2 inceleniyor...
Sayfa 2/2 inceleniyor...
'İnşaat ve Konut' kategorisinde 34 tablo bulundu


In [9]:
# İsme göre tablo seçimi
def get_link_by_name(tables, name):
    for title, link in tables:
        if name == title:
            return link
    return None

# Verilerin Alınması
TÜİK tarafından açıklanan verilerin linkleri *selenium* yardımı ile elde edilmiş ve *pandas* yardımı ile okunarak dataframeler oluşturulmuştur. Bu dataframeler verilerin işlenebilmesi amacıyla düzenlerek kullanıma uygun hale getirilmiştir.

In [10]:
# Linklerin alınması
link_satis_sekli = get_link_by_name(konut_tablolar, "İstatistiksel Tablolar Satış Şekline ve Satış Durumuna Göre Konut Satış Sayıları") #Satış Şekline ve Satış Durumuna Göre Konut Satış Sayıları linki
link_yabancilara = get_link_by_name(konut_tablolar, "İstatistiksel Tablolar Yabancılara Yapılan Konut Satış Sayıları") #Yabancılara Yapılan Konut Satış Sayıları linki
link_uyruklar = get_link_by_name(konut_tablolar, "İstatistiksel Tablolar Ülke Uyruklarına Göre Yabancılara Yapılan Konut Satış Sayıları") #Uyruklarına Göre Yabancılara Yapılan Konut Satış Sayıları linki
link_cinsiyet = get_link_by_name(konut_tablolar, "İstatistiksel Tablolar İllere Göre Cinsiyet Ayrıntısında Konut Satış Sayıları") #İllere Göre Cinsiyet Ayrıntısında Konut Satış Sayıları linki
link_sehirler = get_link_by_name(konut_tablolar, "İstatistiksel Tablolar İllere Göre Konut Satış Sayıları") #İllere Göre Konut Satış Sayıları linki

In [11]:
#Alınan linklerin pandas yardımı ile dataframelere aktarılması
df = pd.read_excel(link_satis_sekli)
time.sleep(15)
df_yabancilar = pd.read_excel(link_yabancilara, usecols="D")
time.sleep(15)
df_uyruklar = pd.read_excel(link_uyruklar, usecols="A:C", skiprows=2, skipfooter=4)
time.sleep(15)
df_cinsiyet = pd.read_excel(link_cinsiyet, usecols="A:B, D:G", skiprows=2, skipfooter=6)
time.sleep(15)
df_sehirler = pd.read_excel(link_sehirler, skiprows=2, skipfooter=5)

In [12]:
#Ana dataframein düzenlenmesi; gereksiz kolon ve satırların çıkaralması boş verilerin doldurulması.
df.drop(columns=["Unnamed: 3", "Unnamed: 5", "Unnamed: 6", "Unnamed: 8", "Unnamed: 9", "Unnamed: 11", "Unnamed: 12", "Unnamed: 14",], inplace=True)
new_header =["Yıl", "Ay", "Toplam Satış Adedi", "İpotekli Satışlar","Diğer Satışlar",  
             "İlk Satışlar", "İkinci El Satışlar" ]
df.columns = new_header

bugun = datetime.date.today()
buyil = bugun.year

df = df[buyil-2009:-13]
df.reset_index(drop=True, inplace=True)
df.fillna(method="ffill", inplace=True)

In [13]:
#Yabancı satış bilgilerinin düzenlenerek ana dataframee eklenmesi
df_yabancilar = df_yabancilar[buyil-2011:-3]
df_yabancilar.reset_index(drop=True, inplace=True)
df["Yabancılara Yapılan Satışlar"] = df_yabancilar.astype(int)

In [14]:
#Yıl ve Ay kolonlarındaki verilerden faydalanarak Dönem kolonunun datetime veri tipinde oluşturulması.
splitter = lambda x: x.split(" - ")[1]
df["Dönem"] = df["Ay"].apply(splitter)
df["Dönem"] = df["Yıl"].astype(str) + " - " + df["Dönem"]
df["Dönem"] = pd.to_datetime(df["Dönem"])

# Verilerin Görselleştirilmesi

Veriler görselleştirilirken, grafiklerin ilk bakışta açıkça anlaşılabilmesi amacıyla verilerin son 24 döneme ait kısımları başlangıç olarak gösterilmiş, kullanıcının geçmiş dönemleri de incelemek istemesine karşılık; grafik geçmişe doğru ilerleyebilir nitelikte tasarlanmıştır.

# Toplam Satış Sayıları

In [15]:
#Son 24 aya ait verilerin öne çıkarılarak grafiğin gösterilmesi
#Grafiğin x eksenini interaktif hale getirmek için başlangıç ve bitiş değerlerinin son 24 dönemi ifade edecek şekilde string olarak bir liste içerisine tanımlanması
range = [(df["Dönem"][-25:-24] + timedelta(days=15)).to_string().split("   ")[1], (df["Dönem"][-1:] + timedelta(days=15)).to_string().split("   ")[1]]
fig = px.bar(df, x="Dönem", y='Toplam Satış Adedi', title="Toplam Konut Satışları")
fig.update_xaxes(dtick="M1", tickformat="%b\n%Y")
fig.update_layout(xaxis_rangeslider_visible=True, xaxis_range=range)
fig.show()

# İpotekli ve Diğer Konut Satış Sayıları

In [16]:
fig = px.bar(df, x="Dönem", y=["İpotekli Satışlar", "Diğer Satışlar"], 
            title="Şatış Şekline Göre Konut Satışları", barmode="group",
            labels={"value": "Adet", "variable": "Satış Şekli"})
fig.update_xaxes(dtick="M1", tickformat="%b\n%Y")
fig.update_layout(xaxis_rangeslider_visible=True, xaxis_range=range)
fig.show()

# İlk ve İkinci El Konut Satış Sayıları

In [17]:
fig = px.bar(df, x="Dönem", y=["İlk Satışlar", "İkinci El Satışlar"], 
             title="Satış Durumuna Göre Konut Satışları", barmode="group",
            labels={"value": "Adet", "variable": "Satış Durumu"})
fig.update_xaxes(dtick="M1", tickformat="%b\n%Y")
fig.update_layout(xaxis_rangeslider_visible=True, xaxis_range=range)
fig.show()

# Yabancılara Yapılan Konut Şatış Sayıları

In [18]:
fig = px.bar(df, x="Dönem", y="Yabancılara Yapılan Satışlar", 
             title="Yabancılara Yapılan Konut Satışları", 
            labels={"value": "Adet", "variable": "Satış Şekli"})
fig.update_xaxes(dtick="M1", tickformat="%b\n%Y")
fig.update_layout(xaxis_rangeslider_visible=True, xaxis_range=range)
fig.show()

In [19]:
fig = px.bar(df, x="Dönem", y=["Yabancılara Yapılan Satışlar", "Toplam Satış Adedi"], 
             title="Toplam Satışlara Oranla Yabancılara Yapılan Konut Satışları", barmode="group",
             labels={"value": "Adet", "variable": "Satış Şekli"})
fig.update_xaxes(dtick="M1", tickformat="%b\n%Y")
fig.update_layout(xaxis_rangeslider_visible=True, xaxis_range=range)
fig.show()

# Uyruklarına Göre Yabancıların Konut Edinim Sayıları

In [20]:
#Uyruklarına göre yabancılara yapılan satışlarda dataframein düzenlenmesi
df_uyruklar.columns = ["Yıl", "Ülke", "Toplam"]
df_uyruklar = df_uyruklar.replace(r'^\s*$', np.nan, regex=True) #Tamammen boş olan hücreleri NaN olarak doldurma
df_uyruklar.fillna(method="ffill", inplace=True)
df_uyruklar["Yıl"] = df_uyruklar["Yıl"].astype(int)
df_uyruklar = df_uyruklar[df_uyruklar["Ülke"].str.contains("Toplam - Total") == False] #Toplam satırlarını dataframeden çıkarma

splitter = lambda x: x.split(" -")[0] #Ülkelerin isimlerini sadeleştirme
df_uyruklar["Ülke"] = df_uyruklar["Ülke"].apply(splitter)

#Verilerin lineplot için hazırlanması
df_uyruklar_line = df_uyruklar[df_uyruklar["Ülke"].str.contains("Diğer ülkeler") == False]
df_uyruklar_line = df_uyruklar_line[(df_uyruklar_line["Yıl"] < buyil)]

In [21]:
fig = px.line(df_uyruklar_line, x="Yıl", y="Toplam", title=f"Uyruklarına Göre Yabancıların Konut Edinimi ({buyil} hariç)", color = "Ülke")
fig.show()

In [22]:
# Uyruğuna göre en fazla satış yapılanların hesaplanması
df_uyruklar_toplam = df_uyruklar.groupby("Ülke").sum().sort_values("Toplam", ascending=False)
df_uyruklar_toplam = df_uyruklar_toplam.drop(columns="Yıl").reset_index()
df_uyruklar_toplam.head()

,Ülke,Toplam
0,Diğer ülkeler,71201.0
1,Irak,51939.0
2,Rusya Federasyonu,50756.0
3,İran,43629.0
4,Suudi Arabistan,16183.0


In [23]:
fig = px.pie(df_uyruklar_toplam, values='Toplam', names='Ülke', title=f"Uyruğuna Göre Yabancıların Konut Edinim Oranları (2015 - {buyil} Toplamı)",
            labels={"value":"Satış Adedi", "variable":"Şehir"}, hole=.3)
fig.update_traces(textposition='inside', textinfo='percent+label')
fig.show()

# Cinsiyete Göre Konut Edinim Sayıları

In [24]:
#İllere göre cinsiyet ayrıntısında yapılan konut satışları dataframeinin düzenlenmesi 
df_cinsiyet.columns=["Yıl", "İl", "Erkek", "Kadın", "Ortak", "Diğer"] 
df_cinsiyet.fillna(method="ffill", inplace=True)
df_cinsiyet.replace(to_replace="-", value="0", inplace=True) # - işareti olan dataların 0 ile değiştirilmesi
df_cinsiyet = df_cinsiyet[df_cinsiyet["İl"].str.contains("Türkiye") == False] #Aradaki toplam satırlarının çıkarılması
df_cinsiyet[["Yıl", "Ortak", "Diğer"]] = df_cinsiyet[["Yıl", "Ortak", "Diğer"]].astype(int) #İlgili kolonlardaki değerlerin sayısal değerlere dönüştürülmesi
df_cinsiyet

,Yıl,İl,Erkek,Kadın,Ortak,Diğer
1,2014,Adana,11809,6630,283,2175
2,2014,Adıyaman,3262,970,23,225
3,2014,Afyonkarahisar,3516,1790,590,364
4,2014,Ağrı,1098,220,5,24
5,2014,Aksaray,2877,1280,93,147
...,...,...,...,...,...,...
897,2024,Uşak,3373,2400,112,266
898,2024,Van,6857,2559,24,275
899,2024,Yalova,6361,4797,313,2928
900,2024,Yozgat,4540,2213,87,259


In [25]:
#Yıl bazında cinsiyete göre yapılan satışların hesaplanması
df_cinsiyet_yıl_toplam = df_cinsiyet.groupby("Yıl").sum().reset_index()
df_cinsiyet_yıl_toplam
#Diğer kolonu tüzel kişiler ile yabancı kişileri ifade etmektedir.

,Yıl,Erkek,Kadın,Ortak,Diğer
0,2014,695727,343209,27919,98526
1,2015,774874,382237,29708,102501
2,2016,801048,391334,29088,119983
3,2017,837928,409453,29178,132755
4,2018,790006,421286,26877,137229
5,2019,775653,420276,24141,128659
6,2020,892347,483007,23424,100538
7,2021,819730,506495,28900,136731
8,2022,821132,484654,24193,155643
9,2023,690438,401933,19610,113945


## Cinsiyete Göre Konut Edinimleri Yıllık
Cinsiyete göre konut edinim sayıları yıllık olarak aşağıdaki grafikte gösterilmektedir. Bu grafikte "Ortak" alanı edinim sahiplerinin en az bir erkek ve bir kadından oluştuğunu, "Diğer" alanı ise edinimin tüzel kişi veya yabancı kişi tarafından gerçekleştirildiğini ifade etmektedir.

In [26]:
fig = px.bar(df_cinsiyet_yıl_toplam, x="Yıl", y=["Erkek", "Kadın", "Diğer", "Ortak"], barmode="group",
             title="Türkiye'de Cinsiyete Göre Konut Edinimi", labels={"value":"Adet", "variable":"Cinsiyet"})
fig.show()

In [27]:
df_toplam = df_cinsiyet_yıl_toplam.drop(columns="Yıl").sum().to_frame().reset_index()
df_toplam.rename(columns={"index":"Cinsiyet", 0:"Toplam Satış"}, inplace=True)

## Cinsiyete Göre Konut Edinim Oranları

In [28]:
fig = px.pie(df_toplam, values="Toplam Satış", names="Cinsiyet",
            title=f"Yapılan Satışların Cinsiyete Göre Dağılımı (2014 - {buyil-1})", hole=.3)
fig.update_traces(textinfo='percent+label') 
fig.show()

In [29]:
df_cinsiyet_il = df_cinsiyet.groupby("İl").sum().reset_index().drop(columns=["Yıl", "Diğer", "Ortak"])
df_cinsiyet_il["Oran"] = df_cinsiyet_il["Kadın"] / (df_cinsiyet_il["Kadın"] + df_cinsiyet_il["Erkek"])
df_cinsiyet_il_enfazla = df_cinsiyet_il.sort_values("Oran", ascending=False).head(5)
df_cinsiyet_il_enaz = df_cinsiyet_il.sort_values("Oran").head(5)

In [30]:
df_cinsiyet_il_enfazla

,İl,Erkek,Kadın,Oran
11,Balıkesir,169019,130398,0.435506
53,Muğla,105015,78655,0.428241
74,Çanakkale,71374,52607,0.424315
71,Yalova,59150,42725,0.419387
9,Aydın,177838,123089,0.409033


## Konut Edinim Oranlarının Birbirine En Yakın Olduğu Beş İl

In [31]:
fig = px.bar(df_cinsiyet_il_enfazla, x="İl", y=["Erkek", "Kadın"], barmode="group",
             title="Edinim Oranlarının Birbirine En Yakın Olduğu 5 İl", labels={"value":"Adet", "variable":"Cinsiyet"})
fig.show()

In [32]:
df_cinsiyet_il_enaz

,İl,Erkek,Kadın,Oran
10,Ağrı,21380,5374,0.200867
54,Muş,17538,4720,0.212059
62,Siirt,23683,6637,0.218898
17,Bitlis,19836,5625,0.220926
39,Kars,24051,7247,0.231548


## Konut Edinim Oranlarının Birbirine En Uzak Olduğu Beş İl

In [33]:
fig = px.bar(df_cinsiyet_il_enaz, x="İl", y=["Erkek", "Kadın"], barmode="group",
             title="Edinim Oranlarının Birbirine En Uzak Olduğu 5 İl", labels={"value":"Adet", "variable":"Cinsiyet"})
fig.show()

# İllere Göre Konut Satış Sayıları

In [34]:
#İllere göre satış sayılarına ait dataframe'in düzenlenmesi
df_sehirler = df_sehirler[buyil-2012:]
df_sehirler.rename(columns={'Yıl                      Year':"Yıl", 'Ay\nMonth':"Ay", 'Toplam      Total ':"Toplam"}, inplace=True)
df_sehirler.fillna(method="ffill", inplace=True)
df_sehirler.reset_index(drop=True, inplace=True)
df_sehirler.replace(to_replace="-", value="0", inplace=True)
df_sehirler["Yıl"] = df_sehirler["Yıl"].astype(int)
df_sehirler["Hakkari"] = df_sehirler["Hakkari"].astype(int)
df_sehirler.drop(columns=["Toplam"], inplace=True)

In [35]:
#Dataframe'deki kolonların yılına ve ayına göre verileri içeren iki kolon olacak şekilde dağıtılması
df_aylik_sehirler = df_sehirler.melt(id_vars=["Yıl", "Ay"])
df_aylik_sehirler.rename(columns={"variable": "Şehir", "value": "Satış Adedi"}, inplace=True)

In [36]:
#Yıl ve Ay kolonlarındaki verilerden faydalanarak Dönem kolonunun datetime veri tipinde oluşturulması.
splitter = lambda x: x.split(" - ")[1]
df_aylik_sehirler["Dönem"] = df_aylik_sehirler["Ay"].apply(splitter)
df_aylik_sehirler["Dönem"] = df_aylik_sehirler["Yıl"].astype(str) + " - " + df_aylik_sehirler["Dönem"]
df_aylik_sehirler["Dönem"] = pd.to_datetime(df_aylik_sehirler["Dönem"])

df_aylik_sehirler

,Yıl,Ay,Şehir,Satış Adedi,Dönem
0,2013,Ocak - January,Adana,1548,2013-01-01
1,2013,Şubat - February,Adana,1667,2013-02-01
2,2013,Mart - March,Adana,1826,2013-03-01
3,2013,Nisan - April,Adana,1875,2013-04-01
4,2013,Mayıs - May,Adana,1941,2013-05-01
...,...,...,...,...,...
11902,2024,Kasım - November,Zonguldak,777,2024-11-01
11903,2024,Aralık - December,Zonguldak,1030,2024-12-01
11904,2025,Ocak - January,Zonguldak,566,2025-01-01
11905,2025,Şubat - February,Zonguldak,543,2025-02-01


İllere göre satışların aylık çizgi grafikte gösterimi, esas görünüşte çok anlamlı olmasa da; grafiğin interaktif olması ve "Şehir" listesinden istenilen şehir çift tıklanarak seçildiğinde o şehre ait verilerin aylık olarak görülebilmesi ve yanına başka şehirlerin de eklenebilir ve kaldırılabilir olması grafiğin kullanılabilirliğini arttırmaktadır.

In [37]:
fig = px.line(df_aylik_sehirler, x="Dönem", y="Satış Adedi", color="Şehir",
             title="İl Bazında Aylık Satış Adetleri")
fig.show()

In [38]:
df_gecenyil = df_sehirler[df_sehirler["Yıl"] == buyil-1]
df_gecenyil = df_gecenyil.groupby("Yıl").sum().reset_index()
df_gecenyil = df_gecenyil.melt(id_vars="Yıl")

In [39]:
fig = px.pie(df_gecenyil, values='value', names='variable', title=f"{buyil-1} Yılı İl Bazında Satış Oranları",
            labels={"value":"Satış Adedi", "variable":"Şehir"}, hole=.3)
fig.update_traces(textposition='inside', textinfo='percent+label')
fig.show()

# Harita
Türkiye'de yapılan konut satışlarının %30'undan fazlası İstanbul, Ankara ve İzmir illerinde yapılmaktadır. Haritanın daha anlamlı görülebilmesi için bu üç büyük ile ait rakamlar haritada gösterilmemiştir.

## Geçen Yıl İl Bazında Yapılan Satışlar

In [40]:
#Haritanın daha anlamlı görünebilmesi için en fazla satış olan üç il çıkarılmıştır.
df_gecenyil_eksik = df_gecenyil.sort_values("value")[:-3]

In [41]:
#Türkiye İller GeoJson verilerinin webden çekilemesi
with urlopen('https://raw.githubusercontent.com/cihadturhan/tr-geojson/master/geo/tr-cities-utf8.json') as response:
    geojson = json.load(response) #Cihad Turhan'a teşekkürler
geojson["features"][2]["properties"] = {'name':'Afyonkarahisar'} #Mevcut verilerle Geojson verilerinin eşleştirilmesi

In [42]:
fig = px.choropleth_mapbox(df_gecenyil_eksik, geojson=geojson, color="value",
                           locations="variable", featureidkey="properties.name",
                           center={"lat": 39, "lon": 35.5},
                           opacity=.6,
                           mapbox_style="carto-positron", zoom=4.5,
                           color_continuous_scale = ["Darkgreen", "Yellow", "Red"],
                           labels={"variable":"Şehir", "value":"Satış Adedi"})
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()